**1* Desafio Mineração de Dados**
Student: Péricles Borges
Class: Knowledge Engineering

References
    ----------
    Original data description document with further info about fields/format:
    http://sorry.vse.cz/~berka/challenge/pkdd1999/berka.htm

In [4]:
import pandas as pd
import pkdd99_database as pkdd99

In [5]:
def get_database():

    account = pd.read_csv('account.csv','delimiter',';')
    card = pd.read_csv('card.csv','delimiter',';')
    client = pd.read_csv('client.csv','delimiter',';')
    disp = pd.read_csv('disp.csv','delimiter',';')
    district = pd.read_csv('district.csv','delimiter',';')
    loan = pd.read_csv('loan.csv','delimiter',';')
    order = pd.read_csv('order.csv','delimiter',';')
    trans = pd.read_csv('trans.csv','delimiter',';',low_memory=False)

    account.name = 'Account'
    card.name = 'Card'
    client.name = 'Client'
    disp.name = 'Disp'
    district.name = 'District'
    loan.name = 'Loan'
    order.name = 'Order'
    trans.name = 'Trans'

    # descriptions are cut/pasted from the Financial Data Description webpage for the data
    account.description = 'each record describes static characteristics of an account'
    card.description = 'each record describes a credit card issued to an account'
    client.description = 'each record describes characteristics of a client'
    disp.description = 'each record relates together a client with an account'
    district.description = 'each record describes demographic characteristics of a district'
    loan.description = 'each record describes a loan granted for a given account'
    order.description = 'each record describes characteristics of a payment order'
    trans.description = 'each record describes one transaction on an account'

    account.notes = '(one account can have one or more clients, e.g. married couples)'
    card.notes = '(one account can have one or more credit cards)'
    client.notes = '(one client can have one or more accounts)'
    disp.notes = '(disposition connects a client/account pair and allows to link one or more cards)'
    district.notes = '(neighborhoods for both bank/account branches and client homes.  same 16 fields as original A1-16.)'
    loan.notes = '(one account may have zero or one loan.  see loan.codes for ABCD status definitions)'
    order.notes = '(one payment order is from one account)'
    trans.notes = '(category,bank,account are NaN for some types/operations.)'
    loan.codes = 'Loan status codes:\nA = contract finished, no problems\nB = contract finished, loan not payed\nC = running contract, OK so far\nD = running contract, client in debt'

    account,card,client,disp,district,loan,order,trans = _translate_and_clean(
        account,card,client,disp,district,loan,order,trans
    )

    return account,card,client,disp,district,loan,order,trans

In [11]:
def table_summary(df):

    print(df.name,': length',len(df),':',df.description)
    print(df.notes)
    print(df.head(3))
    print(' ')

TypeError: table_summary() missing 1 required positional argument: 'df'